# Cricket Prediction using machine learning

## Data Preprocessing

In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv('ipl.csv',index_col=0)

In [3]:
dataset = dataset.drop(columns=['gender', 'match_type','date','umpire_1','umpire_2','player of the match','win_by_runs','win_by_wickets'])

In [4]:
# columns with missing values
dataset.columns[dataset.isnull().any()]

Index(['city'], dtype='object')

In [5]:
# replace missing column with mode value
dataset['city'].fillna(dataset['city'].mode()[0], inplace=True)

In [6]:
dataset.columns[dataset.isnull().any()]

dataset.replace(['Mumbai Indians','Kolkata Knight Riders','Royal Challengers Bangalore','Deccan Chargers','Chennai Super Kings',
                 'Rajasthan Royals','Delhi Daredevils','Gujarat Lions','Kings XI Punjab',
                 'Sunrisers Hyderabad','Rising Pune Supergiants','Kochi Tuskers Kerala','Pune Warriors','Rising Pune Supergiant']
                ,['MI','KKR','RCB','DC','CSK','RR','DD','GL','KXIP','SRH','RPS','KTK','PW','RPS'],inplace=True)

In [7]:
dataset.head(5)

,city,team 1,team 2,toss_decision,toss_winner,venue,winner
0,Bangalore,RCB,KKR,field,RCB,M Chinnaswamy Stadium,KKR
1,Chandigarh,KXIP,CSK,bat,CSK,"Punjab Cricket Association Stadium, Mohali",CSK
2,Delhi,DD,RR,bat,RR,Feroz Shah Kotla,DD
3,Mumbai,MI,RCB,bat,MI,Wankhede Stadium,RCB
4,Kolkata,KKR,DC,bat,DC,Eden Gardens,KKR


In [8]:
def createDict(series) :
    
    dictionary={}
    
    i=0
    
    for ser in series :
        if(ser in dictionary) :
            continue
        dictionary[ser]=i
        i=i+1
        
    return dictionary

In [9]:
teamDict=createDict(dataset['team 1'])

cityDict=createDict(dataset['city'])

venueDict=createDict(dataset['venue'])

tossDecisionDict=createDict(dataset['toss_decision'])

winnerDict=dict(teamDict)

winnerDict['tie']=14

winnerDict['no result']=15

In [10]:
encode = {
'team 1': teamDict,
'team 2': teamDict,
'toss_winner': teamDict,
'winner': winnerDict,
'city':cityDict,
'venue':venueDict,
'toss_decision': tossDecisionDict    
 }
dataset.replace(encode, inplace=True)

In [12]:
def prediction(Model,X_train,y_train,X_test,y_test) :
    
    clf=Model()
    
    clf.fit(X_train,y_train)
    
    print(clf.score(X_test,y_test))
    
    return clf

In [13]:
def predictWinner():    
    
    from sklearn.neural_network import MLPClassifier

    from sklearn.svm import LinearSVC

    from sklearn.linear_model import LogisticRegression

    from sklearn.ensemble import RandomForestClassifier

    clf_A = prediction(MLPClassifier,X_train,y_train,X_test,y_test)

    clf_B = prediction(LinearSVC,X_train,y_train,X_test,y_test)

    clf_C = prediction(LogisticRegression,X_train,y_train,X_test,y_test)

    clf_D = prediction(RandomForestClassifier,X_train,y_train,X_test,y_test)
    

In [14]:
from sklearn.neural_network import MLPClassifier

from sklearn.linear_model import LogisticRegression

def buildModel(dataset,team1,team2) :

    
    dataset=dataset[
        ((dataset['team 1']==team1)&(dataset['team 2']==team2) | 
         (dataset['team 1']==team2)&(dataset['team 2']==team1))
    ]


    winner = dataset['winner']

    features = dataset.drop('winner',axis=1)

    features = pd.get_dummies(features)

    clf=LogisticRegression()

    clf.fit(features,winner)

    return clf

In [23]:
def getPrediction(city,team1,team2,toss_decision,toss_winner,venue) :

    predictionSet = pd.DataFrame({
        'city':cityDict[city],
        'team 1':teamDict[team1],
        'team 2':teamDict[team2],
        'toss_decision':[toss_decision],
        'toss_winner':teamDict[toss_winner],
        'venue':venueDict[venue]
    })

    predictionSet = pd.get_dummies(predictionSet)
    
    clf=buildModel(dataset,teamDict[team1],teamDict[team2])
    
    prediction=clf.predict(predictionSet)
    
    for key,value in teamDict.items() :
        
        if(value==prediction) :
            
            print(key)

In [24]:

getPrediction('Bangalore','KKR','RCB','field','RCB','M Chinnaswamy Stadium')

KKR


In [25]:
getPrediction('Chandigarh','KXIP','CSK','bat','CSK','Punjab Cricket Association Stadium, Mohali')

CSK


In [26]:
getPrediction('Delhi','DD','RR','bat','RR','Feroz Shah Kotla')

DD
